# Baseline delays and rates

[L. Blackburn, Sep 2018; rewritten for Python 3.9, Dec 2022]

The purpose of this test is to check continuity of delay and rate solutions. Fringe delay and rate should be relatively smooth from scan to scan, and outliers indicate some issue with the fringe solution.

In [ ]:
# basic import and helper functions
import pandas as pd
from eat.io import hops, util
from eat.hops import util as hu
from eat.plots import util as pu
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import seaborn as sns

sns.reset_orig()
# sns.set_palette(sns.color_palette(sns.hls_palette(16, l=.6, s=.6)))
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# %config InlineBackend.figure_formats=['svg']

nb_stdout = sys.stdout # grab for later

def wide(w=8, h=3): plt.setp(plt.gcf(), figwidth=w, figheight=h); \
    plt.tight_layout()

def tightx(): plt.autoscale(enable=True, axis='x', tight=True)

def multline(xs, fun=plt.axvline):
    for x in xs: fun(x, alpha=0.25, ls='--', color='k')

def toiter(x):
    return(x if hasattr(x, '__iter__') else [x,])

# pd.options.display.float_format = '{:,.6f}'.format
from IPython.display import display, HTML
display(HTML("<style>"
    + "#notebook { padding-top:0px !important; } " 
    + ".container { width:100% !important; } "
    + ".end_space { min-height:0px !important; } "
    + "</style>"))

In [ ]:
# define and load data
alistf = "alist.v6"
datadir = os.environ['DATADIR']

a = util.noauto(hops.read_alist(os.path.join(datadir, alistf)))

# Pre-process alist dataframe
util.fix(a)
util.unwrap_mbd(a)
util.add_days(a)
util.add_delayerr(a)
util.add_path(a)
util.add_scanno(a)

In [ ]:
# data filters -- remove SMAR-SMAW baselines (only applicable to EHT2017 data)
thres = 7.0
a = a[(a.snr > thres) & ~a.baseline.isin({'RS', 'SR'})].copy()

In [ ]:
# Compute the boundaries between expt_nos
sorted_a = a.sort_values(['expt_no', 'scan_no'])
last_scans = sorted_a.groupby('expt_no')['scan_no'].max() # Find the 'max' scan_no for each expt_no
elines = (last_scans.iloc[:-1] + 0.5).to_numpy() # Drop the final expt_no and offset by 0.5

In [ ]:
# Plotting the delays by iterating over sites
sites = sorted(set(''.join(a.baseline)))
for (i, site) in enumerate(sites):
    hu.trendplot(a, site, vlines=elines, col='mbd_unwrap')
    plt.title('measured delays [ns @ %.0f MHz]' % (a.iloc[0].ref_freq))
    wide(12, 4)
    plt.show()

In [ ]:
# Plotting the rates by iterating over sites
for (i, site) in enumerate(sites):
    hu.trendplot(a, site, vlines=elines, col='delay_rate')
    plt.title('measured rate [ps/s @ %.0f MHz]' % (a.iloc[0].ref_freq))
    wide(12, 4)
    plt.show()